In [41]:
# import pandas as pd
# df = pd.read_csv("EURUSD_Candlestick_1_D_ASK_05.05.2003-30.06.2021.csv")

# #Check if NA values are in data
# df=df[df['volume']!=0]
# df.reset_index(drop=True, inplace=True)
# df.isna().sum()
# df.tail()

import pandas as pd
!pip install pandas_ta
import pandas_ta as ta
import yfinance as yf
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from datetime import datetime
import matplotlib.pyplot as plt

## Get data from NVDA
df = yf.download('NVDA','2014-1-1','2024-4-30', auto_adjust=True)
df.columns = [x.lower() for x in df.columns]

#Check if NA values are in data
df=df[df['volume']!=0]
df.reset_index(drop=True, inplace=True)
df.isna().sum()
df['RSI'] = ta.rsi(df.close, length=14)

df.tail()

zsh:1: command not found: pip


[*********************100%%**********************]  1 of 1 completed


,open,high,low,close,volume,RSI
2593,807.690002,827.690002,802.640015,824.229980,43855900,45.676400
2594,839.500000,840.820007,791.830017,796.770020,51220800,42.009676
2595,788.679993,833.229980,782.229980,826.320007,42464100,46.945406
2596,838.179993,883.309998,833.869995,877.349976,55101100,54.195689
2597,875.950012,879.919983,852.659973,877.570007,38897100,54.224736


In [42]:
wick_threshold = 0.0005

## detect initial support fractal candle
def support(df1, l, n1, n2): #n1 n2 before and after candle l
    """
    the candle l should have a low lower than all neighboring candles --> support canlde
    detect a low fractal candle
    df: datframe
    l: current candle index to be tested if its a support or resistance candle
    n1: number of candles before
    n2: number of candles after to be compared with
    """
    if (df1.low[l - n1:l].min() < df1.low[l] or
        df1.low[l + 1:l + n2 + 1].min() < df1.low[l]):
        return 0

    ## select candles showing a strong rejection movement
    ## a candle wick exceeding a certain threshold can be considered a strong rejection movement
    candle_body = abs(df1.open[l] - df1.close[l])
    lower_wick = min(df1.open[l], df1.close[l]) - df1.low[l]
    if (lower_wick > candle_body) and (lower_wick > wick_threshold):
        return 1

    return 0
##detect resistance candles
def resistance(df1, l, n1, n2): #n1 n2 before and after candle l
    """
    the candle l should have a high greater than all neighboring candles
    detect a high fractal candle --> resistance candle aka pivot point
    """
    if ( df1.high[l-n1:l].max() > df1.high[l] or
       df1.high[l+1:l+n2+1].max() > df1.high[l] ):
        return 0

    candle_body = abs(df1.open[l]-df1.close[l])
    upper_wick = df1.high[l]-max(df1.open[l], df1.close[l])
    if (upper_wick > candle_body) and (upper_wick > wick_threshold) :
        return 1

    return 0

In [43]:
def closeResistance(l, levels, lim, df):
    """
    purpose: detect if a candle is close enough to S/R and contained within the S/R levels
    taking into account defined conditions:
    1. wick should be close to S/R levels 2. wick should be contained by S/R
    levels: the prices where the levels are, they key levels found
    limit: the threshold for measuring closeness, if candlewick is closed to one of these levels
    """

    if len(levels) == 0:
        return 0


    ## the highest wick should be within a threshold distance of the highest R level
    c1 = abs(df.high[l] - min(levels, key=lambda x:abs(x - df.high[l]))) <= lim

    ## body top (open or close)'s difference from the high of the R level should be within a limit
    ## alternatively body should be contained within the level
    c2 = abs( max(df.open[l], df.close[l]) - min(levels, key=lambda x:abs(x-df.high[l])) ) <= lim

    ##whichever one is lower (open or close) should be less than the (min) high of the R candles
    c3 = min(df.open[l], df.close[l]) < min(levels, key=lambda x:abs(x-df.high[l]))

    ## low of the candle should be lower than the (min) high of the R candles
    c4 = df.low[l] < min(levels, key=lambda x:abs(x - df.high[l]))

    ### either one of the two c1 or c2 should meet the criteria
    if( (c1 or c2) and c3 and c4):
        return min(levels, key=lambda x:abs(x - df.high[l]))
    else:
        return 0

def closeSupport(l,levels,lim, df):

    if len(levels) == 0:
        return 0

    ## candle's low should be within a threshold distance of the candle with lowest closest to the current candle
    c1 = abs(df.low[l] - min(levels, key=lambda x:abs(x - df.low[l]))) <= lim

    ## candle's minimum (either open or close) should be within a threshold distance of the min level support candle
    c2 = abs(min(df.open[l],df.close[l]) - min(levels, key=lambda x:abs(x - df.low[l]))) <= lim

    ## the max of either open or close should be greater than the minimum of all support candles
    c3 = max(df.open[l], df.close[l]) > min(levels, key=lambda x:abs(x - df.low[l]))

    ## the high of the candle should be greater than the min of all S candles
    c4 = df.high[l] > min(levels, key=lambda x:abs(x-df.low[l]))

    if( (c1 or c2) and c3 and c4 ):
        return min(levels, key=lambda x:abs(x-df.low[l]))
    else:
        return 0

In [44]:

"""
3rd condition: prev candles should also be contained within the S/R levels
l: index of the current candle
level_backCandles: number of back Candles
level: level we are testing
"""


def is_below_resistance(l, level_backCandles, level, df):
    return df.loc[l - level_backCandles:l - 1, 'high'].max() < level

def is_above_support(l, level_backCandles, level, df):
    return df.loc[l - level_backCandles:l - 1, 'low'].min() > level

In [45]:
def check_candle_signal(l, n1, n2, backCandles, df):
    """
    takes the index of the current candle, and check if the number of candles
    are contained within the levels
    l: current candle index
    n1, n2: candles on the left and right
    backCandles: number of backCandles contained within the key level

    """
    ss = [] # support levels
    rr = [] # resistance levels

    ##

    ## merge all levels that are close to each other
    for subrow in range(l - backCandles, l - n2):
        if support(df, subrow, n1, n2):
            ss.append(df.low[subrow])
        if resistance(df, subrow, n1, n2):
            rr.append(df.high[subrow])

    ss.sort() #keep lowest support when popping a level
    for i in range(1, len(ss)):
        if(i >= len(ss)):
            break
        if abs(ss[i] - ss[i-1]) <= 0.0001: # merging close distance levels
            ss.pop(i)

    rr.sort(reverse=True) # keep highest resistance when popping one
    for i in range(1,len(rr)):
        if(i >= len(rr)):
            break
        if abs(rr[i]-rr[i-1])<=0.0001: # merging close distance levels
            rr.pop(i)

    #----------------------------------------------------------------------
    # joined levels
    # the same level can exist as support and resistance depending upon where the price is heading
    rrss = rr+ss
    rrss.sort()
    for i in range(1,len(rrss)):
        if(i>=len(rrss)):
            break
        if abs(rrss[i]-rrss[i-1])<=0.0001: # merging close distance levels
            rrss.pop(i)
    cR = closeResistance(l, rrss, 150e-5, df)
    cS = closeSupport(l, rrss, 150e-5, df)
    ## how close should the candle be so it exists within the zone of the support/resistance -- 150e-5
    #----------------------------------------------------------------------

    # cR = closeResistance(l, rr, 150e-5, df)
    # cS = closeSupport(l, ss, 150e-5, df)
    # could we consider the average RSI for the trend momentum?
    ### if we have a candle that is close to resistance and its neighbors are below the resistance and at the same time
    ### the RSI of the preceding candle is below 45 then we return 1 which is a bearish signal: we have a downtrend
    ### momentum bc RSI is below 45
    if (cR and is_below_resistance(l, 6, cR, df) and df.RSI[l-1:l].min() < 35): # and df.RSI[l]>65
        # 1 is a bearish signal, we are approaching resistance level from below, approaching downtrend
        return 1
    elif(cS and is_above_support(l, 6, cS, df) and df.RSI[l-1:l].max() > 65 ):#and df.RSI[l]<35
        # 2 is a bullish signal
        return 2
    else:
        return 0


In [46]:
from tqdm import tqdm

n1 = 8 # left preceding candles
n2 = 6 # forward preceding candles

## for each candle we are looking at 140 hours/5days before
## (enough to establish sufficiient resistance levels for that week)
backCandles = 140

signal = [0 for i in range(len(df))]

for row in tqdm(range(backCandles + n1, len(df) - n2)):
    signal[row] = check_candle_signal(row, n1, n2, backCandles, df)

df["signal"] = signal


100%|███████████████████████████████████████| 2444/2444 [00:34<00:00, 70.38it/s]


In [47]:
df[df['signal']==1].count()

open      0
high      0
low       0
close     0
volume    0
RSI       0
signal    0
dtype: int64

In [48]:
print(df.columns)

df.columns = ['Open', 'High', 'Low', 'Close', 'Volume', 'RSI', 'signal']
df

Index(['open', 'high', 'low', 'close', 'volume', 'RSI', 'signal'], dtype='object')


,Open,High,Low,Close,Volume,RSI,signal
0,3.754697,3.768848,3.707528,3.740546,26009200,NaN,0
1,3.747623,3.754698,3.683944,3.695736,25933200,NaN,0
2,3.733472,3.773566,3.698095,3.745264,40949200,NaN,0
3,3.783001,3.820737,3.757058,3.806586,33328800,NaN,0
4,3.820736,3.877339,3.806585,3.858472,30819200,NaN,0
...,...,...,...,...,...,...,...
2593,807.690002,827.690002,802.640015,824.229980,43855900,45.676400,0
2594,839.500000,840.820007,791.830017,796.770020,51220800,42.009676,0
2595,788.679993,833.229980,782.229980,826.320007,42464100,46.945406,0
2596,838.179993,883.309998,833.869995,877.349976,55101100,54.195689,0


In [49]:
def SIGNAL():
    return df.signal

In [50]:
from backtesting import Strategy

class MyCandlesStrat(Strategy):  
    def init(self):
        super().init()
        #TODO: look up self.I
        self.signal1 = self.I(SIGNAL)

    def next(self):
        super().next() 
        if self.signal1 == 2:
            self.buy()
            #TODO: use StopLoss and TakeProfit later
        elif self.signal1 == 1:
            self.position.close()

In [51]:
from backtesting import Backtest

bt = Backtest(df, MyCandlesStrat, cash=10_000, commission=.002)
stat = bt.run()
stat

/var/folders/hc/_w7g8hhx6y34zl23s8tbcn2w0000gn/T/ipykernel_31325/2007398463.py:3: UserWarning: Data index is not datetime. Assuming simple periods, but `pd.DateTimeIndex` is advised.
  bt = Backtest(df, MyCandlesStrat, cash=10_000, commission=.002)


Start                                     0.0
End                                    2597.0
Duration                               2597.0
Exposure Time [%]                         0.0
Equity Final [$]                      10000.0
Equity Peak [$]                       10000.0
Return [%]                                0.0
Buy & Hold Return [%]            23361.012221
Return (Ann.) [%]                         0.0
Volatility (Ann.) [%]                     NaN
Sharpe Ratio                              NaN
Sortino Ratio                             NaN
Calmar Ratio                              NaN
Max. Drawdown [%]                        -0.0
Avg. Drawdown [%]                         NaN
Max. Drawdown Duration                    NaN
Avg. Drawdown Duration                    NaN
# Trades                                  0.0
Win Rate [%]                              NaN
Best Trade [%]                            NaN
Worst Trade [%]                           NaN
Avg. Trade [%]                    

In [52]:
bt.plot()

/usr/local/lib/python3.12/site-packages/backtesting/_plotting.py:659: UserWarning: found multiple competing values for 'toolbar.active_drag' property; using the latest value
  fig = gridplot(
/usr/local/lib/python3.12/site-packages/backtesting/_plotting.py:659: UserWarning: found multiple competing values for 'toolbar.active_scroll' property; using the latest value
  fig = gridplot(


GridPlot(id='p1398', ...)